In [18]:
! pip install pyodbc
! pip install sqlalchemy
! pip install python-slugify

import pandas as pd
import json
import pyodbc
import sqlalchemy as db
import urllib
from slugify import slugify

# exec(open("azure_credentials.py").read())

# database_name = "AID2023"
# driver='{ODBC Driver 17 for SQL Server}'
# server_name = "aid-2023-srv.database.windows.net"

# server = 'tcp:{server_name},1433'.format(server_name = server_name)

# #create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

# connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
#     driver=driver,
#     server=server,
#     database=database_name,
#     username=username,
#     password=password
# )                              

# cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
# cursor: pyodbc.Cursor = cnxn.cursor() 

In [111]:
#Inseram tabelul "school_details" si facem un dataframe

In [95]:
file_path = "school_details.json"

with open(file_path) as json_file:
    school_details = json.load(json_file)
    
data = pd.DataFrame(school_details)
data

,id,idSchool,sfName,sfLevel,studyFormationType,educationType,fundingForm,educationForm,teachingMode,teachingLanguage,techingType,studentsNo
0,930621,11269910,B ÃŽngeraÈ™ii,PreÈ™colar,Grupa micÄƒ,MasÄƒ,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
1,930620,11269910,A Ursuletii,PreÈ™colar,Grupa micÄƒ,MasÄƒ,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
2,930623,11269910,A,PreÈ™colar,Grupa mare,MasÄƒ,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
3,930622,11269910,A,PreÈ™colar,Grupa mijlocie,MasÄƒ,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
4,828033,11263948,A,Gimnazial,Clasa a VI-a,Special,Buget,Zi,Pedagogia curativÄƒ,Limba romÃ¢nÄƒ,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,834477,11256974,C,Gimnazial,Clasa a VII-a,Special,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
996,834455,11256974,B,Primar,Clasa I,Special,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
997,834457,11256974,A,Primar,Clasa a II-a,Special,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0
998,834469,11256974,A,Primar,Clasa pregÄƒtitoare,Special,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ,Normal,0


In [90]:
#Ne asiguram ca baza de date nu are valori lipsa

In [96]:
data.isnull().sum()

id                    0
idSchool              0
sfName                0
sfLevel               0
studyFormationType    0
educationType         0
fundingForm           0
educationForm         0
teachingMode          0
teachingLanguage      0
techingType           0
studentsNo            0
dtype: int64

In [91]:
#Ne asiguram ca baza de date nu are duplicate

In [97]:
duplicates =data[data.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, idSchool, sfName, sfLevel, studyFormationType, educationType, fundingForm, educationForm, teachingMode, teachingLanguage, techingType, studentsNo]
Index: []


In [92]:
#Schimbam denumirile variabilelor "sfLevel" si "studyFormationType" pentru a fi mai usor prelucrarea lor

In [98]:
data = data.rename(columns = {"sfLevel": "schoolLevel"})
data = data.rename(columns = {"studyFormationType": "Class"})

In [94]:
#Stergem coloanele pe care nu le folosim in analiza

In [99]:
data.drop(columns = ["sfName", "techingType", "studentsNo", 'educationType'], inplace = True)

In [112]:
#Modificam typos initiale din coloana "SchoolLevel"

In [100]:
school_levels = {
    'PreÈ™colar':'Prescolar',
    'Gimnazial':'Gimnazial',
    'Profesional':'Profesional',
    'Primar':'Primar',
    'Liceal':'Liceal',
    'Postliceal':'Postliceal',
    'AntepreÅŸcolar': 'Anteprescolar'
}

data['schoolLevel'] = data['schoolLevel'].apply(lambda x: school_levels[x])

In [113]:
#Modificam typos initiale din coloana "Class"

In [101]:
data['Class'].unique()
class_corection = {
    'Grupa micÄƒ':'Grupa mica',
    'Clasa pregÄƒtitoare':'Clasa pregatitoare'
}
data['Class'] = data['Class'].apply(lambda x: class_corection[x] if x in class_corection else x)

In [114]:
#Modificam typos initiale din coloana "FundingForm"

In [102]:
data['fundingForm'].unique()
funding_form = {
    'TaxÄƒ':'Taxa',
    'MixtÄƒ':'Mixta'
}
data['fundingForm'] = data['fundingForm'].apply(lambda x: funding_form[x] if x in funding_form else x)

In [115]:
#Modificam typos initiale din coloana "EducationForm"

In [103]:
data['educationForm'].unique()
education_forms = {
    'Normal':'Normal',
    'Zi':'Zi',
    'Prelungit': 'Prelungit',
    'SÄƒptÄƒmÃ¢nal':'Saptamanal',
    'FrecvenÈ›Äƒ redusÄƒ':'Frecventa redusa',
    'Seral': 'Seral'
}

data['educationForm'] = data['educationForm'].apply(lambda x: education_forms[x])

In [104]:
data

,id,idSchool,schoolLevel,Class,fundingForm,educationForm,teachingMode,teachingLanguage
0,930621,11269910,Prescolar,Grupa mica,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ
1,930620,11269910,Prescolar,Grupa mica,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ
2,930623,11269910,Prescolar,Grupa mare,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ
3,930622,11269910,Prescolar,Grupa mijlocie,Sponsorizare,Normal,TradiÈ›ional,Limba romÃ¢nÄƒ
4,828033,11263948,Gimnazial,Clasa a VI-a,Buget,Zi,Pedagogia curativÄƒ,Limba romÃ¢nÄƒ
...,...,...,...,...,...,...,...,...
995,834477,11256974,Gimnazial,Clasa a VII-a,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ
996,834455,11256974,Primar,Clasa I,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ
997,834457,11256974,Primar,Clasa a II-a,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ
998,834469,11256974,Primar,Clasa pregatitoare,Buget,Zi,TradiÈ›ional,Limba romÃ¢nÄƒ


In [81]:
data["teachingMode"].unique()

array(['TradiÈ›ional', 'Pedagogia curativÄƒ'], dtype=object)

In [116]:
#Modificam typos initiale din coloana "TeachingMode"

In [82]:
teaching_types = {
    'TradiÈ›ional':'Traditional',
    'Pedagogia curativÄƒ':'Pedagogia curativa',
}

data["teachingMode"] = data["teachingMode"].apply(lambda x:teaching_types[x] if x in teaching_types else x)

In [83]:
data

,id,idSchool,schoolLevel,Class,fundingForm,educationForm,teachingMode,teachingLanguage
0,930621,11269910,Prescolar,Grupa mica,Sponsorizare,Normal,Traditional,Limba romÃ¢nÄƒ
1,930620,11269910,Prescolar,Grupa mica,Sponsorizare,Normal,Traditional,Limba romÃ¢nÄƒ
2,930623,11269910,Prescolar,Grupa mare,Sponsorizare,Normal,Traditional,Limba romÃ¢nÄƒ
3,930622,11269910,Prescolar,Grupa mijlocie,Sponsorizare,Normal,Traditional,Limba romÃ¢nÄƒ
4,828033,11263948,Gimnazial,Clasa a VI-a,Buget,Zi,Pedagogia curativa,Limba romÃ¢nÄƒ
...,...,...,...,...,...,...,...,...
995,834477,11256974,Gimnazial,Clasa a VII-a,Buget,Zi,Traditional,Limba romÃ¢nÄƒ
996,834455,11256974,Primar,Clasa I,Buget,Zi,Traditional,Limba romÃ¢nÄƒ
997,834457,11256974,Primar,Clasa a II-a,Buget,Zi,Traditional,Limba romÃ¢nÄƒ
998,834469,11256974,Primar,Clasa pregatitoare,Buget,Zi,Traditional,Limba romÃ¢nÄƒ


In [105]:
data["teachingLanguage"].unique()

array(['Limba romÃ¢nÄƒ', 'Limba maghiarÄƒ'], dtype=object)

In [117]:
#Modificam typos initiale din coloana "TeachingLanguage"

In [106]:
teaching_languages = {
    'Limba romÃ¢nÄƒ':'Limba romana',
    'Limba maghiarÄƒ':'Limba maghiara',
    'Limba ucraineanÄƒ': 'Limba ucraineana',
    'Limba croatÄƒ': 'Limba croata',
    'Limba slovacÄƒ': 'Limba slovaca',
    'Limba englezÄƒ': 'Limba engleza',
    'Limba italianÄƒ': 'Limba italiana',
    'Limba polonÄƒ': 'Limba polona',
    'Limba sÃ¢rbÄƒ': 'Limba sarba',
    'Limba neogreacÄƒ': 'Limba neogreaca',
    'Limba germanÄƒ': 'Limba germana',
    'Limba ceha': 'Limba ceha',
    'Limba rusÄƒ': 'Limba rusa',
    'Limba bulgarÄƒ': 'Limba bulgara',
    'Limba romani (rromani)': 'Limba romani (rromani)'
}

data["teachingLanguage"] = data["teachingLanguage"].apply(lambda x:teaching_languages[x])

In [86]:
data

,id,idSchool,schoolLevel,Class,fundingForm,educationForm,teachingMode,teachingLanguage
0,930621,11269910,Prescolar,Grupa mica,Sponsorizare,Normal,Traditional,Limba romana
1,930620,11269910,Prescolar,Grupa mica,Sponsorizare,Normal,Traditional,Limba romana
2,930623,11269910,Prescolar,Grupa mare,Sponsorizare,Normal,Traditional,Limba romana
3,930622,11269910,Prescolar,Grupa mijlocie,Sponsorizare,Normal,Traditional,Limba romana
4,828033,11263948,Gimnazial,Clasa a VI-a,Buget,Zi,Pedagogia curativa,Limba romana
...,...,...,...,...,...,...,...,...
995,834477,11256974,Gimnazial,Clasa a VII-a,Buget,Zi,Traditional,Limba romana
996,834455,11256974,Primar,Clasa I,Buget,Zi,Traditional,Limba romana
997,834457,11256974,Primar,Clasa a II-a,Buget,Zi,Traditional,Limba romana
998,834469,11256974,Primar,Clasa pregatitoare,Buget,Zi,Traditional,Limba romana


In [110]:
#Inseram tabelul "tempo_jud"

In [107]:
file_path = "tempo_jud.json"

with open(file_path) as json_file:
    tempo_jud = json.load(json_file)
    
data1 = pd.DataFrame(tempo_jud)
data1

,Judet,Elevi_cu_BAC,Ateliere_scolare,Laboratoare,Echipamente,Terenuri_sport,Imigranti
0,Bihor,2915,108,253,6137,73,300
1,Bistrita-Nasaud,1553,47,104,3814,21,177
2,Cluj,3653,112,368,14700,75,692
3,Maramures,2449,67,135,6815,37,326
4,Satu Mare,1460,74,139,3722,39,205
5,Salaj,1171,51,134,2122,17,66
6,Alba,1817,82,162,4882,37,215
7,Brasov,2668,97,203,4005,40,634
8,Covasna,847,93,70,1842,16,66
9,Harghita,1422,99,134,3795,27,93


In [ ]:
#Ne asiguram ca baza de date nu are valori lipsa

In [108]:
data1.isnull().sum()

Judet               0
Elevi_cu_BAC        0
Ateliere_scolare    0
Laboratoare         0
Echipamente         0
Terenuri_sport      0
Imigranti           0
dtype: int64